<a href="https://colab.research.google.com/github/jfjoung/AI_For_Chemistry/blob/main/notebooks/week7/Week_7_Molecule_generation_(SMILES-LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 🎯 **학습 목표**

- **생성 모델(Generative Model)의 개념**과 **지도 학습(Supervised Learning)과의 차이점**을 이해한다.  
- **SMILES 형식의 화합물 표현 방법**을 복습하고, **SMILES 데이터 전처리 방법**을 익힌다.  
- **순환 신경망(RNN)과 LSTM(Long Short-Term Memory)**의 구조와 특징을 학습한다.  
- SMILES 생성을 위한 **LSTM 모델 구성 및 훈련 과정**을 실습한다.  
- **샘플링 및 생성된 분자의 유효성 검증** 과정을 통해 생성 모델의 성능을 평가한다.  
- 간단한 실습을 통해 **화학 데이터 기반의 생성 모델 워크플로우**에 대한 전반적인 이해를 구축한다.


In [ ]:
# 사용할 유틸리티 함수들이 정의된 utils.py 파일을 다운로드
!wget https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/data/week6/utils.py -O utils.py

# 사전 학습된 RNN 모델 파일 다운로드
!wget https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/data/week6/pretrained.rnn.pth -O pretrained.rnn.pth

!git clone https://github.com/rociomer/dl-chem-101.git


In [ ]:
# need to install the RNN repository as a package
%cd dl-chem-101
%cd 03_gen_SMILES_LSTM
!pip install -e .

In [ ]:
%cd ../..

In [ ]:
!pip install torch==2.1
!pip install rdkit
!pip install numpy==1.26

In [ ]:
# 🔄 ***이제 노트북의 런타임을 반드시 재시작해야 합니다!***

# ✅ 왜 재시작해야 하나요?
# - pip으로 새 패키지를 설치하거나, 기존 패키지를 다른 버전으로 설치하면 Colab 환경에 즉시 반영되지 않습니다.
# - 특히 `pip install -e .` 같은 editable 설치는 런타임을 재시작해야 적용됩니다.
# - 또한 패키지나 모듈 경로 설정 충돌을 방지하기 위해 깨끗한 환경에서 시작하는 것이 중요합니다.

# 🔧 어떻게 재시작하나요?
# 메뉴에서 [런타임] → [세션 다시 시작] 클릭
# 또는 단축키: Ctrl+M → . (Windows) / Cmd+M → . (Mac)

# 📌 재시작 후에는 "맨 위 셀부터" 순서대로 모든 코드를 다시 실행해야 합니다.
# 설치된 패키지, 다운로드한 파일, 변수 등이 모두 초기화되었기 때문입니다.

# 생성적 SMILES 모델: 상세 설명

이 Jupyter 노트북은 `REINVENT`에서 가져온 생성적 SMILES 모델 코드베이스를 기반으로 구성되어 있습니다. `REINVENT`에 대한 자세한 내용은 다음 논문들을 참고하세요:

* https://jcheminf.biomedcentral.com/articles/10.1186/s13321-017-0235-x

* https://pubs.acs.org/doi/full/10.1021/acscentsci.7b00512

* https://pubs.acs.org/doi/10.1021/acs.jcim.0c00915

이 노트북에서는 생성적 SMILES 모델의 각 구성 요소가 어떻게 결합되어 작동하는지를 단계별로 설명할 것입니다.

In [ ]:
# 정규 표현식을 위한 모듈
import re

# 수치 계산을 위한 모듈 (벡터, 행렬 연산 등)
import numpy as np

# PyTorch 라이브러리 불러오기
import torch
from torch import nn  # 신경망 구성에 필요한 모듈 (레이어 등)
import torch.nn.functional as F  # 자주 쓰이는 함수들 (예: 활성화 함수, 손실 함수 등)

import warnings
# 특정 경고 메시지(예: 사용 중단 예정 경고)를 무시하도록 설정
warnings.filterwarnings('ignore')


## SMILES 어휘(Vocabulary): 어떻게 생성되는가?

모델의 `Vocabulary` 클래스는 SMILES 문자열을 토큰(token) 시퀀스로 변환하는 역할을 한다.  
토큰은 문자열에서 정의된 가장 작은 의미 단위이며, 더 이상 나눌 수 없는 단위이다.  
자연어처리(Natural Language Processing, NLP)에서는 일반적으로 단어나 문자 수준에서 토큰을 정의한다.  
토큰화를 어떤 수준으로 하느냐에 따라 모델이 출력할 수 있는 단위가 결정된다. 예를 들어 문자(character) 수준으로 토큰화를 하면, 모델은 개별 문자를 출력하게 된다.

생성적 SMILES 모델에서는 문자 수준의 토큰화를 사용하며, 각 토큰은 대체로(unique하지 않을 수 있으므로 ***loosely***) 하나의 원자(atom) 유형에 대응된다.  
단, 괄호(예: "(")는 분기를 나타내는 기호이므로 원자에 대응되지 않고, 연결 구조(connectivity) 정보를 제공한다.


In [ ]:
# SMILES 문자열을 토큰화하고 어휘 사전을 관리하는 클래스들 불러오기
from smiles_lstm.model.smiles_vocabulary import Vocabulary, SMILESTokenizer

`Vocabulary` 클래스의 메서드를 다시 작성해야 합니다 (이 부분은 걱정하지 않아도 됩니다).

In [ ]:
# Union은 타입 힌트에서 여러 타입 중 하나를 허용할 때 사용 (여기선 dict 또는 None 허용)
from typing import Union

class Vocabulary:
    """
    토큰과 그에 대응하는 vocabulary 인덱스를 저장하는 클래스
    """

    def __init__(self, tokens : Union[dict, None]=None, starting_id : int=0) -> None:
        # 토큰과 인덱스를 저장할 딕셔너리 초기화
        self._tokens = {}
        # 현재 인덱스를 저장 (새 토큰 추가 시 이 값이 증가함)
        self._current_id = starting_id

        # 초기 토큰 사전이 주어진 경우, 각각을 vocabulary에 추가
        if tokens:
            for token, idx in tokens.items():
                self._add(token, idx)  # 토큰과 인덱스를 매핑
                # 현재 인덱스를 가장 큰 값 다음으로 업데이트
                self._current_id = max(self._current_id, idx + 1)

    def __getitem__(self, token_or_id : str) -> int:
        # 토큰 또는 인덱스를 주면 그에 대응하는 값(인덱스 또는 토큰)을 반환
        return self._tokens[token_or_id]

    def add(self, token : str) -> int:
        """
        새로운 토큰을 vocabulary에 추가
        """
        # 입력이 문자열인지 확인
        if not isinstance(token, str):
            raise TypeError("Token is not a string")  # 문자열이 아니면 에러 발생
        # 이미 존재하는 토큰이면 해당 인덱스를 반환
        if token in self:
            return self[token]
        # 새 토큰이면 현재 인덱스로 추가
        self._add(token, self._current_id)
        self._current_id += 1  # 다음 인덱스로 업데이트
        return self._current_id - 1  # 추가한 토큰의 인덱스를 반환

    def update(self, tokens : list) -> list:
        """
        여러 개의 토큰을 한꺼번에 vocabulary에 추가
        """
        # 각 토큰을 add 함수로 추가하고 인덱스 리스트를 반환
        return [self.add(token) for token in tokens]

    def __delitem__(self, token_or_id : Union[str, int]) -> None:
        # 주어진 토큰 또는 인덱스를 vocabulary에서 삭제
        other_val = self._tokens[token_or_id]  # 대응되는 값 (인덱스 또는 토큰)
        del self._tokens[other_val]  # 매핑 삭제
        del self._tokens[token_or_id]

    def __contains__(self, token_or_id : Union[str, int]) -> None:
        # 토큰 또는 인덱스가 vocabulary에 존재하는지 여부 반환
        return token_or_id in self._tokens

    def __eq__(self, other_vocabulary : "Vocabulary") -> int:
        # 두 Vocabulary 객체가 같은 내용을 갖는지 비교
        return self._tokens == other_vocabulary._tokens  # pylint: disable=W0212

    def __len__(self) -> int:
        # 총 토큰 수 반환 (토큰:인덱스, 인덱스:토큰 쌍이 모두 들어 있으므로 2로 나눔)
        return len(self._tokens) // 2

    def encode(self, tokens : list) -> np.ndarray:
        """
        토큰 리스트를 vocabulary 인덱스 배열로 인코딩
        """
        vocab_index = np.zeros(len(tokens), dtype=int)  # 0으로 채워진 정수 배열 생성
        for i, token in enumerate(tokens):
            vocab_index[i] = self._tokens[token]  # 각 토큰을 인덱스로 변환
        return vocab_index  # numpy 배열 반환

    def decode(self, vocab_index : np.ndarray) -> list:
        """
        인덱스 배열을 토큰 리스트로 디코딩
        """
        tokens = []
        for idx in vocab_index:
            tokens.append(self[idx])  # 각 인덱스를 대응되는 토큰으로 변환
        return tokens  # 토큰 리스트 반환

    def _add(self, token : str, idx : int) -> None:
        # 새로운 토큰과 인덱스 쌍을 vocabulary에 추가
        if idx not in self._tokens:
            self._tokens[token] = idx  # 토큰 → 인덱스 매핑
            self._tokens[idx] = token  # 인덱스 → 토큰 매핑
        else:
            raise ValueError("IDX already present in vocabulary")  # 중복 인덱스면 에러

    def tokens(self) -> list:
        """
        vocabulary에서 토큰(문자열)만 리스트로 반환
        """
        return [t for t in self._tokens if isinstance(t, str)]  # 문자열인 key만 추출


## 언더스코어(_)로 시작하는 변수의 의미

Python에서 변수 이름 앞에 밑줄(`_`)을 붙이는 것은 해당 변수가 **"내부적으로 사용되는 변수임을 나타내는 관례(convention)"**입니다.  
이는 해당 속성이나 메서드가 **외부에서 직접 접근되기보다는 클래스 내부에서만 사용되기를 의도**한다는 것을 뜻합니다.  
즉, **비공개(private)** 속성처럼 취급되지만, 실제로 접근이 막히는 것은 아니며 단지 사용자가 주의하라는 의미입니다.

예를 들어:

```python
self._tokens = {}
```

이 코드는 `_tokens`가 `Vocabulary` 클래스의 내부 구현 세부사항이며, 외부에서 직접 접근하지 말고  
가능하면 제공된 메서드(`add`, `encode`, `decode` 등)를 통해 다루라는 의도를 담고 있습니다.

👉 정리:

- `_변수명` : 내부 전용 (비공식적인 private), 외부 사용자가 직접 접근하지 않는 것이 좋음  
- `__변수명` : name mangling이 적용되어 서브클래스에서 실수로 접근하지 못하도록 함 (보다 강한 보호)  
- `_` 단독 사용 : 반복문에서 사용하지 않는 변수 이름으로 자주 쓰임 (`for _ in range(10):` 등)
- `__이름__` : **매직 메서드(magic method)** 또는 **덩더 메서드(dunder method)** 라고 부르며,  
  파이썬이 내부적으로 특정 동작을 처리하기 위해 호출하는 특별한 메서드나 속성 이름임  
  (예: `__init__`, `__len__`, `__getitem__`, `__str__` 등).  
  사용자가 직접 호출하기보다는 파이썬 인터프리터에 의해 자동으로 호출됨.

먼저 SMILES 문자열이 어떻게 토큰화되는지 살펴보겠습니다.  
이 과정은 **데이터 준비(Data Preparation)**에 해당하며, 생성적 SMILES 모델을 훈련하기 위한 첫 번째 단계입니다.  
이 코드에서 가장 중요한 부분은 SMILES를 ***어떻게(how)*** 토큰화할지를 정의하는 ***규칙(rules)***입니다.  
이 규칙은 **정규 표현식(regular expressions, regexp)**을 기반으로 하며, 특정 패턴을 인식하여 매칭합니다.  
아래 코드는 `SMILESTokenizer` 클래스의 일부입니다.

정규 표현식(regular expressions, regexp)은 문자열에서 **특정 패턴을 찾거나 분리하기 위해 사용하는 표현 언어**입니다.  
SMILES 토큰화에서는 각 문자가 고유한 화학적 의미를 가지므로, 복잡한 규칙을 단순하게 표현할 수 있는 정규 표현식이 매우 유용합니다.

아래는 SMILES 토큰화에 사용된 정규 표현식의 간단한 문법 설명입니다:

- `[...]` : 대괄호 안의 내용을 **그대로 하나의 토큰**으로 인식합니다. 예: `[nH]`, `[C@H]`
  - `\[` 와 `\]` 는 대괄호 자체를 문자로 인식하기 위한 이스케이프 처리입니다.
  - `[^]]*` 는 닫는 대괄호(`]`)가 아닌 모든 문자를 0개 이상 포함한다는 의미입니다.
- `%\d{2}` : `%` 기호로 시작하고 **두 자리 숫자**가 따라오는 형태를 찾습니다. 예: `%10`, `%12`
  - `\d` 는 숫자(digit)를 의미하고, `{2}`는 두 자리 숫자라는 의미입니다.
- `(Br|Cl)` : `Br` 또는 `Cl` 중 **하나에 해당하는 경우**를 인식합니다.
  - `|` 기호는 **또는(or)** 을 의미하며, 괄호로 그룹핑하여 하나의 단위로 처리합니다.

이러한 정규 표현식들을 순서대로 적용하면, SMILES 문자열을 화학적으로 의미 있는 **토큰 단위로 정확하게 분해**할 수 있습니다.


In [ ]:
# SMILES 토큰화를 위한 정규 표현식(regexp) 사전 정의
REGEXPS = {
    "brackets": re.compile(r"(\[[^\]]*\])"),  # 대괄호 내부 원자(예: [nH], [C@H])를 하나의 토큰으로 분리
    "2_ring_nums": re.compile(r"(%\d{2})"),   # 10 이상의 ring closure 번호 (%10, %11 등)를 하나의 토큰으로 분리
    "brcl": re.compile(r"(Br|Cl)")            # 'Br'과 'Cl'을 두 글자지만 하나의 토큰으로 인식하도록 분리
}

# 위에서 정의한 정규 표현식을 적용할 순서
REGEXP_ORDER = ["brackets", "2_ring_nums", "brcl"]

def tokenize(data, with_begin_and_end=True):
    """SMILES 문자열을 토큰화하는 함수"""

    # 재귀적으로 정규 표현식을 적용하여 문자열을 토큰 리스트로 분할
    def split_by(data, regexps):
        if not regexps:
            return list(data)  # 더 이상 정규식이 없으면 남은 문자열을 문자 단위로 분할
        regexp = REGEXPS[regexps[0]]  # 현재 적용할 정규 표현식 선택
        splitted = regexp.split(data)  # 정규식 기준으로 문자열 분할
        tokens = []
        for i, split in enumerate(splitted):
            if i % 2 == 0:
                # 짝수 index: 정규식에 매칭되지 않은 부분 → 다음 정규식으로 재귀적으로 처리
                tokens += split_by(split, regexps[1:])
            else:
                # 홀수 index: 정규식에 매칭된 부분 → 그대로 하나의 토큰으로 추가
                tokens.append(split)
        return tokens

    # 정규 표현식들을 순서대로 적용하여 토큰 리스트 생성
    tokens = split_by(data, REGEXP_ORDER)

    # 시작("^")과 끝("$") 토큰을 추가하여 문장의 경계를 표시
    # 이는 생성 모델 학습 시 시작/종료를 명확히 알려주는 역할을 함
    if with_begin_and_end:
        tokens = ["^"] + tokens + ["$"]

    return tokens  # 최종 토큰 리스트 반환


**REGEXPS** 변수는 SMILES 문자열을 어떤 패턴으로 토큰화할지를 정의하는 정규 표현식들을 저장하고 있습니다.  
각 패턴과 그 화학적 근거를 하나씩 설명해보겠습니다. 아래는 **"brackets"** 패턴을 시각적으로 나타낸 예시입니다 (*debuggex를 사용해 생성됨*):


<p align="middle">
  <img src="https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/img/REGEXPS.png" width="500"/>
</p>

이 정규 표현식 패턴은 대괄호(`[]`) 안에 있는 문자들을 하나의 토큰으로 묶습니다.  
SMILES 문법에서 대괄호는 **전하가 있는 원자나 특수한 표기**를 나타내는 데 사용됩니다.  
예를 들어, 양전하를 띤 질소 원자는 `[N+]` 와 같이 대괄호 안에 표현됩니다.


<p align="middle">
  <img src="https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/img/CNHC.png" width="150"/>
</p>

이제 실제로 SMILES 문자열을 토큰화해보겠습니다.


In [ ]:
charged_nitrogen_smiles = 'C[NH+]C'

# [NH+]는 하나의 토큰으로 간주되며, 생성 모델의 어휘(vocabulary)에 포함됨
# 따라서 생성 모델은 양전하를 띤 질소 원자를 출력하는 법을 학습하게 됨

# "with_begin_and_end"는 토큰화된 SMILES 앞뒤에 시작(^)과 종료($) 토큰을 추가할지를 지정
# 여기서는 토큰화만 보여주기 위한 목적이므로 False로 설정함

print(tokenize(charged_nitrogen_smiles, with_begin_and_end=False))  # SMILES 문자열을 토큰 단위로 출력


이제 나머지 두 개의 정규 표현식 패턴에 대해서도 같은 방식으로 설명해보겠습니다.  
아래는 `"2_ring_nums"` 패턴의 시각적 예시입니다:

<p align="middle">
  <img src="https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/img/REGEXPS2.png" width="500"/>
</p>


SMILES 문자열에서 등장하는 `"숫자"`는 일반적으로 **명시적 수소 원자** 또는 **고리(ring) 닫힘**을 의미합니다.  
예를 들어 "CH₃"는 탄소에 수소 3개가 결합되어 있다는 것을 의미하지만, 대부분의 경우 수소는 **암시적으로(implicitly)** 표현됩니다.

이 정규 표현식은 **고리 닫힘 번호**와 관련이 있습니다.  
화합물의 고리들은 **숫자로 구분**되며, 고리가 10개 이상인 경우에는 **퍼센트 기호("%")를 붙여서** 표현합니다.  
예를 들어, "%10"은 분자 내의 10번째 고리를 의미합니다.

따라서 이 정규식 패턴은 `"두 자리 수까지의 고리 번호"`를 인식하도록 설계되었습니다.  
예: `%10`, `%11`, `%23` 등

극단적인 예시로 아래에 있는
**버크민스터풀러렌(Buckminsterfullerene)** 또는 **"버키볼(bucky ball)"** 의 SMILES의 토큰화해보겠습니다.


<p align="middle">
  <img src="https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/img/bucky_ball.png" width="200"/>
</p>


In [ ]:
# SMILES 문자열에 숫자가 포함되어 있는 것을 주목
bucky_ball_smiles = 'C12=C3C4=C5C6=C1C7=C8C9=C1C%10=C%11C(=C29)C3=C2C3=C4C4=C5C5=C9C6=C7C6=C7C8=C1C1=C8C%10=C%10C%11=C2C2=C3C3=C4C4=C5C5=C%11C%12=C(C6=C95)C7=C1C1=C%12C5=C%11C4=C3C3=C5C(=C81)C%10=C23'

# 토큰화된 결과에는 "%10", "%11" 등과 같은 **두 자리 ring closure 숫자**들이 있으며,
# 이들은 정규 표현식에 따라 **하나의 독립된 토큰**으로 처리됨
print(tokenize(bucky_ball_smiles, with_begin_and_end=False))  # SMILES를 시작/종료 토큰 없이 순수하게 토큰화하여 출력


간단히 덧붙이자면, 이처럼 매우 큰 분자들은 일반적으로 학습 데이터에서 제외됩니다.  
이는 해당 분자들이 합리적인 크기의 의약품 후보로 보기에는 지나치게 크기 때문입니다  
(물론 예외도 존재합니다).


<p align="middle">
  <img src="https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/img/REGEXPS3.png" width="500"/>
</p>


마지막 정규 표현식 패턴은 `"brcl"`입니다:

이 패턴은 단순히 **브로민(Br)** 과 **염소(Cl)** 를 각각 **하나의 고유한 토큰**으로 인식하게 만듭니다.  
이러한 할로겐 원자들은 **의약품 설계에서 매우 자주 등장하는 구성 요소**입니다.  

아래는 브로민이 포함된 분자(bromoethane)의 예시입니다:


<p align="middle">
  <img src="https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/img/CCBr.png" width="200"/>
</p>


In [ ]:
bromoethane_smiles = 'CCBr'

# 예상대로 "Br"은 두 글자이지만 하나의 토큰으로 처리됨
# 이는 "brcl" 정규 표현식이 'Br'과 'Cl'을 개별 토큰으로 인식하도록 정의되어 있기 때문

print(tokenize(bromoethane_smiles, with_begin_and_end=False))  # 시작/종료 토큰 없이 SMILES 문자열을 토큰화하여 출력


## 생성 모델의 SMILES 어휘(Vocabulary) 구성하기

이전 섹션에서는 SMILES 문자열이 어떻게 토큰화되는지를 살펴보았습니다.  
이제 이 토큰화된 SMILES를 실제로 사용하기 위해서는 `Vocabulary`를 생성해야 합니다.  

이는 마치 영어와 같은 자연어에서 어휘력이 말할 수 있는 문장의 범위를 결정하듯,  
**분자 생성 모델의 vocabulary는 생성할 수 있는 원자 종류**, 즉 **어떤 분자를 만들 수 있는지를 결정**합니다.  
이 문맥에서 "문장(sentence)"은 곧 "분자(molecule)"를 의미합니다.

아래의 메서드는 `vocabulary.py` 파일에 정의되어 있습니다.


In [ ]:
def create_vocabulary(smiles_list, tokenizer):
    """SMILES 문법을 위한 어휘(vocabulary)를 생성하는 함수"""

    tokens = set()  # 중복 없이 토큰을 모으기 위한 집합(set) 생성

    # 주어진 모든 SMILES 문자열에 대해 반복
    for smi in smiles_list:
        # 각 SMILES 문자열을 토큰화하여 토큰 집합에 추가
        # 시작(^), 종료($) 토큰은 여기서는 제외 (나중에 수동으로 추가)
        tokens.update(tokenize(data=smi, with_begin_and_end=False))

    # Vocabulary 클래스 인스턴스 생성
    vocabulary = Vocabulary()

    # 어휘에 특수 토큰("^", "$")를 먼저 추가한 뒤, 알파벳 순으로 정렬된 토큰들을 추가
    # "$"는 끝(end) 토큰이며, 일반적으로 패딩 토큰과 같은 인덱스를 가짐 (index 0)
    vocabulary.update(["$", "^"] + sorted(tokens))

    return vocabulary  # 완성된 vocabulary 객체 반환


In [ ]:
# 예제 데이터셋: 앞에서 사용한 3개의 분자를 포함하고 있음
# "버키볼(bucky ball)"도 포함되어 있는데, 실제 의약품 분자로는 적합하지 않지만
# 너무 큰 분자가 데이터셋에 포함될 경우 어떤 영향을 미치는지 설명하기 위해 사용됨
smiles_dataset = [
    'C[NH+](C)C',  # 양전하를 띤 질소가 포함된 분자
    'C12=C3C4=C5C6=C1C7=C8C9=C1C%10=C%11C(=C29)C3=C2C3=C4C4=C5C5=C9C6=C7C6=C7C8=C1C1=C8C%10=C%10C%11=C2C2=C3C3=C4C4=C5C5=C%11C%12=C(C6=C95)C7=C1C1=C%12C5=C%11C4=C3C3=C5C(=C81)C%10=C23',  # bucky ball
    'CCBr'  # 브로모에테인 (Br 포함)
]

# 위 데이터셋을 기반으로 vocabulary 생성
# SMILES 문자열을 토큰화한 후, 전체 데이터에서 사용되는 고유한 토큰을 수집하여 Vocabulary 객체로 생성
vocabulary = create_vocabulary(smiles_list=smiles_dataset, tokenizer=SMILESTokenizer)


이제 예제 SMILES 데이터셋을 사용하여 `Vocabulary` 객체를 생성했습니다.  
이제 이 객체를 조금 더 자세히 살펴보겠습니다:

In [ ]:
print(f'There are {len(vocabulary)} unique tokens in the vocabulary.\n')
print(f'The unique tokens are: \n{vocabulary.tokens()}')

`Vocabulary`에 20개의 토큰이 포함되어 있다는 것은, 생성 모델이 이 20개의 토큰만을 출력할 수 있다는 의미입니다.  
이 토큰들의 목록은 위에서 확인할 수 있습니다.  

예제 데이터셋에 "버키볼(bucky ball)"을 포함한 이유는,  
**매우 큰 분자들이 "희귀한(rare)" 토큰을 포함함으로써 vocabulary의 크기를 증가시킬 수 있다는 점을 보여주기 위해서**입니다.  
뿐만 아니라, 버키볼은 일반적으로 생성하고자 하는 분자의 종류가 아니며, "약물 유사성(drug-likeness)"도 낮습니다.

**여기서 중요한 점은, vocabulary가 커질수록 생성 모델의 속도가 느려진다는 것입니다.**  
물론 우리가 매우 큰 분자를 실제로 생성하고자 한다면 이는 허용 가능한 일이지만,  
**신약 개발(drug discovery)**에서는 다음과 같은 두 가지 이유로 이러한 분자들을 종종 데이터셋에서 제거합니다:

1) **데이터셋 내 모든 SMILES가 "합리적인" 분자임을 보장**합니다.  
   이것은 생성 모델이 어떤 분자들을 생성할지를 결정하는 데 영향을 미칩니다.  
   왜냐하면 ***생성 모델을 학습할 때의 목표는, 주어진 데이터셋의 분포를 따르는 새로운 분자들을 생성하는 것이기 때문입니다.***  
   이에 대해서는 이후 "모델 세부 설명" 섹션에서 자세히 다룰 예정입니다.

2) **모델 속도를 저하시킬 수 있는 "희귀 토큰"을 제거**합니다.  
   "희귀 토큰"이란, 학습 데이터셋 내에 등장 빈도가 매우 낮은 토큰들을 의미합니다.  
   이런 토큰들은 모델 학습 및 예측 속도에 부정적인 영향을 줄 수 있으며,  
   이에 대한 구체적인 이유는 역시 이후 "모델 세부 설명" 섹션에서 설명됩니다.


마지막으로, SMILES 시퀀스를 구성하는 토큰들이 머신러닝 모델에 입력되기 위해서는  
**수치적(numerical) 표현**으로 변환되어야 합니다.  

이 작업은 `Vocabulary` 클래스에서 수행되며,  
각 고유한 토큰을 **정수 인덱스(numerical index)**에 매핑(mapping)합니다.  

아래는 그 예시입니다:


In [ ]:
# 브로모에테인은 SMILES로 "CCBr"로 표현됨
bromoethane_smiles = 'CCBr'

# 앞서 정의된 토큰화 함수를 사용하여 SMILES 문자열을 토큰 리스트로 변환
# 시작(^) 및 종료($) 토큰은 제외하고 순수하게 SMILES만 토큰화함
tokenized_bromoethane = tokenize(bromoethane_smiles, with_begin_and_end=False)

# 토큰화된 결과 출력
tokenized_bromoethane


위에서 생성한 고유 토큰 목록과 비교해보면, "Br"은 인덱스 17번에, "C"는 인덱스 18번에 위치해 있는 것을 확인할 수 있습니다.  

이제 이를 직접 확인해보겠습니다:


In [ ]:
# Vocabulary 클래스의 "encode" 메서드는 토큰 리스트를 받아서 각각에 대응되는 정수 인덱스를 반환함
# 여기서는 'C', 'C', 'Br' → [18, 18, 17] 과 같이 변환되는 것이 기대됨
vocabulary.encode(tokenized_bromoethane)

이러한 수치적(numerical) 표현이 생성 모델에서 ***어떻게*** 사용되는지는 `RNN Sampling: Step-by-Step` 섹션에서 자세히 설명할 예정입니다.  

우선, SMILES를 처리하는 **순환 신경망(Recurrent Neural Network, RNN)**이 무엇을 의미하는지부터 알아보겠습니다.


## SMILES를 사용하는 순환 신경망(RNN)이란?

분자 생성 모델은 **순환 신경망(Recurrent Neural Network, RNN)**을 사용하며, 이 RNN은 학습 데이터셋 내의 **SMILES 시퀀스들이 따르는 확률 분포를 학습**하는 역할을 합니다.  
아래 섹션에서는 이 개념이 정확히 무엇을 의미하는지 하나씩 풀어보겠습니다.

### 핵심 개념

- 우리의 목표는 모델이 **유효한(valid)** SMILES를 생성할 수 있도록 하는 것입니다.  
  유효한 SMILES란, 예를 들어 RDKit(Python 기반의 케미인포매틱스 라이브러리)을 통해  
  분자의 2차원 구조로 변환 가능한 문자열을 의미합니다.

- **목표**: 다음 단계들을 통해 RNN에게 SMILES 문법(syntax)을 학습시킵니다.

1) SMILES 데이터셋(훈련 데이터)을 준비합니다.  
   예를 들어 ChEMBL(https://www.ebi.ac.uk/chembl/) 또는 ZINC(https://zinc.docking.org/)와 같은    공개 데이터베이스에는 "drug-like"한 분자들이 대량으로 수록되어 있습니다.

2) 이 SMILES 데이터셋을 RNN에 입력하고, RNN은 이 SMILES들을 **재현(reproduce)**하는 것을 학습합니다.

**참고:** 이것이 바로 "훈련 데이터셋 내 SMILES 시퀀스가 따르는 확률 분포를 학습한다"는 의미입니다.  
RNN은 한 번에 하나의 토큰을 출력하며, 훈련 데이터의 SMILES를 성공적으로 재현해내는 과정이  
곧 SMILES 문법을 학습하는 과정입니다.  
반대로, 학습되지 않은 RNN은 화학적 의미가 없는 **무작위 토큰 시퀀스**만 생성하게 됩니다.

---

Marwin Segler의 논문 [https://pubs.acs.org/doi/full/10.1021/acscentsci.7b00512] 에서도 제시되었던 구체적인 예시로
다음과 같은 토큰 시퀀스를 생성했다고 가정해봅시다:

**c1ccccc**

이 시점에서 모델이 다음 토큰으로 **"1"**을 출력할 확률이 높다면, 다음과 같은 전체 시퀀스를 얻게 됩니다:

**c1ccccc1**

이 SMILES 문자열은 **벤젠(benzene)**을 나타냅니다:

<!-- <div align="middle"> -->
<img src="https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/img/benzene.png" width="100"/>
<!-- </div> -->

벤젠은 생물학적으로 활성이 있는 분자들에서 매우 흔히 나타나며,  
다음 토큰으로 "1"을 정확히 예측했다는 것은 모델이 **조건적 의존성(conditional dependence)**을 학습했다는 것을 보여줍니다.  

즉, 모델이 다음 토큰을 예측할 때 지금까지 관찰한 시퀀스를 기반으로 결정해야 한다는 뜻입니다.  
이전에 등장한 토큰들의 조합이 **어떤 토큰이 그 다음에 와야 하는지에 대한 단서**를 줄 수 있기 때문입니다.

이러한 개념은 다음 섹션에서 보다 수학적으로 정리됩니다.


### 수식으로 보는 문제 정의 (Formal Problem Set-up)

특정 SMILES 시퀀스를 생성할 확률은 다음과 같이 정의됩니다:

$$P_\theta(S) = P_\theta(s_1) * \prod_{t = 2}^{T} P_\theta(s_t|s_{t-1}, s_{t-2},...,s_1)$$

여기서 $S$는 SMILES 시퀀스, $s_t$는 개별 토큰, $\theta$는 모델의 학습 가능한 파라미터를 의미합니다.  
위 수식은 다음과 같은 의미를 담고 있습니다:  
특정한 SMILES 시퀀스를 생성할 확률은  
- 첫 번째 토큰이 생성될 확률  
- 그 다음에, 첫 번째 토큰이 주어졌을 때 두 번째 토큰이 생성될 확률  
- 그 다음에, 첫 번째와 두 번째 토큰이 주어졌을 때 세 번째 토큰이 생성될 확률  
…  
이러한 조건부 확률(***conditional probabilities***)들의 곱으로 표현된다는 것입니다.

여기서 $P_\theta(s_t | s_{<t})$ 형태의 확률은 **지금까지 생성된 토큰들($s_1, s_2, ..., s_{t-1}$)을 입력으로 받았을 때,  
다음 토큰 $s_t$이 생성될 확률**을 의미합니다.  

이 확률값은 파라미터 $\theta$를 갖는 모델이 출력하는 결과이며,  
실제로는 RNN의 마지막 출력층에서 **softmax 함수를 통해 vocabulary 전체에 대한 확률 분포**를 계산하게 됩니다.  
그중 특정 토큰 $s_t$에 해당하는 확률값이 바로 $P_\theta(s_t | s_{<t})$입니다.


이러한 조건 의존성을 모델링하기 위해 RNN이 사용됩니다.  
RNN은 ***숨겨진 상태 행렬(hidden state matrices)***을 통해
지금까지 생성된 토큰들의 정보를 기억하고, 그 정보를 기반으로 다음 토큰을 예측합니다.  
이러한 구조 덕분에 RNN은 ***조건적 의존성(conditional-dependence)***을 효과적으로 반영할 수 있습니다.

RNN의 전체 구조에 대한 설명은 생략하지만,  
일반적인 피드포워드 신경망(FFNN)과 비교하여 어떻게 조건 의존성을 반영하는지를  
다음 수식으로 간략히 비교해볼 수 있습니다.

---

**일반 피드포워드 신경망(FFNN)의 연산:**

$$H = \phi(XW_1 + b_1)$$  
$$Output = HW_2 + b_2$$

- $\phi$는 활성화 함수 (activation function)  
- $X$는 입력 행렬  
- $W_1$은 가중치 행렬, $b_1$은 편향  
- 첫 번째 수식은 hidden layer를 계산하고, 두 번째 수식은 최종 출력 계산

---

**RNN의 연산:**

$$H_t = \phi(X_tW_{input} + H_{t-1}W_{hidden} + b_1)$$  
$$Output = H_tW_{output} + b_2$$

FFNN과 달리, RNN은 $H_{t-1}$이라는 항이 추가됩니다.  
여기서 한 번 토큰을 생성하는 것을 하나의 **시간 단계(time-step)**라고 하면,  
- 현재 시점의 hidden state $H_t$는  
  입력 $X_t$와 **이전 시점의 hidden state** $H_{t-1}$을 모두 고려하여 계산됩니다.  
따라서 RNN은 시간에 따라 순차적으로 생성된 토큰들의 문맥(context)을 반영할 수 있습니다.

---

**참고:**  
실제 이 모델에서 사용되는 순환 셀(recurrent cell)은 `Long Short-Term Memory (LSTM)`이라는 구조입니다.  
LSTM은 위 수식보다 더 복잡한 연산을 수행하지만,  
핵심은 여전히 이전 시점의 hidden state ($H_{t-1}$)를 활용하여  
현재 상태를 계산한다는 점에서 동일합니다.  
LSTM의 구체적인 수식과 구조에 대해서는 아래 원 논문을 참고하세요:  
https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.676.4320&rep=rep1&type=pdf

---

마지막으로, RNN의 출력(즉, 다음에 어떤 토큰을 생성할지 결정)은  
전체 `Vocabulary`에 대해 정의된 ***다항 확률 분포(multinomial probability distribution)***입니다.  
즉, 모든 가능한 토큰에 대한 조건부 확률을 출력하게 됩니다.  
이 부분은 다음 섹션에서 실제로 살펴볼 예정입니다.

## RNN 샘플링: 단계별 과정 (Step-by-Step)

RNN을 이용한 샘플링은 다음과 같은 단계로 이루어집니다:

1) **토큰 임베딩 벡터(token embedding vector)**를 얻는다

2) 이 임베딩 벡터를 **RNN에 입력**으로 전달한다

3) RNN의 출력을 기반으로 **softmax 조건부 확률**을 계산한다

이제 각 단계를 하나씩 살펴보겠습니다.


### 임베딩 (Embedding)

이 섹션에서는 토큰의 **수치적 표현(numerical representation)**이  
RNN의 입력으로 사용되는 **임베딩 벡터(embedding vector)**로  
***어떻게 변환되는지(how)***에 대해 설명합니다.


In [ ]:
# 브로모에테인의 SMILES 표기법은 "CCBr"
bromoethane_smiles = 'CCBr'

# 앞에서 정의한 tokenize 함수를 이용해 SMILES 문자열을 토큰 리스트로 변환
# 시작(^) 및 종료($) 토큰은 생략하고 순수한 SMILES만 토큰화함
tokenized_bromoethane = tokenize(bromoethane_smiles, with_begin_and_end=False)

# 토큰화된 결과 확인
tokenized_bromoethane


### 임베딩 레이어란? 왜 원-핫 인코딩보다 유리한가?

임베딩 레이어는 토큰의 **숫자 인덱스(integer index)**를  
모델이 학습 가능한 **고정 길이 벡터(embedding vector)**로 변환해주는 층(layer)입니다.  

각 토큰은 학습 초기에 무작위로 초기화된 벡터를 하나씩 가지고 있으며,  
모델이 학습되면서 해당 벡터들도 함께 업데이트되어 각 토큰의 의미적/구조적 특성을 학습하게 됩니다.

SMILES 생성 모델에서는 각 토큰(C, Br, [, %, ...)이 정수 인덱스로 표현되고,  
이 인덱스가 임베딩 레이어를 거치면 **RNN이 이해할 수 있는 벡터 형태의 입력**으로 변환됩니다.

토큰을 벡터로 표현하는 가장 직관적인 방법은 **원-핫 인코딩(one-hot encoding)**입니다.  
그러나 생성 모델에서는 일반적으로 **임베딩(embedding)** 표현이 더 효율적이고 효과적입니다.

예를 들어, 아래는 토큰 'Br'이 vocabulary에서 인덱스 17번일 때의 두 가지 표현 방식입니다:

- **원-핫 인코딩 (one-hot vector, 길이 20)**  
  `[0, 0, 0, ..., 0, 1, 0]` ← 17번째 위치만 1이고 나머지는 모두 0

- **임베딩 벡터 (embedding vector, 길이 5)**  
  예: `[-1.1590, -0.1173,  0.1660,  0.3528, -1.1199]`  
  (임베딩은 학습 가능한 실수 벡터이며, 의미를 반영함)

---

### 🔍 차이점 정리

| 항목               | 원-핫 인코딩                  | 임베딩 벡터                          |
|------------------|----------------------------|------------------------------------|
| 벡터 길이          | vocabulary 크기만큼 (예: 20) | 보통 작고 고정된 차원 (예: 5차원)     |
| 값의 형태          | 대부분 0, 특정 위치만 1      | 실수 값으로 구성된 밀집(dense) 벡터 |
| 의미적 관계 표현     | 불가능                      | 가능 (학습을 통해 유사성 반영 가능)   |
| 파라미터로 학습 여부 | ❌ (고정됨)                  | ✅ (RNN과 함께 학습됨)               |

---

따라서, 임베딩 벡터는 **차원을 줄이고**,  
**토큰 간의 의미적 유사성**도 반영할 수 있어  
분자 생성 모델처럼 **시퀀스를 기반으로 학습하는 모델**에 훨씬 적합합니다.



In [ ]:
# "임베딩 레이어(Embedding layer)"를 생성
# - num_embeddings: vocabulary에 있는 총 토큰 수 (여기서는 20개)
# - embedding_dim: 각 토큰을 표현할 벡터의 차원 수 (여기서는 5차원 벡터)
embedding_layer = nn.Embedding(num_embeddings=20,
                               embedding_dim=5)

# 임베딩 레이어의 weight를 출력하여 각 인덱스에 해당하는 임베딩 벡터들을 확인
# 인덱스 17번과 18번(뒤에서 세 번째, 두 번째 벡터)은 뒤쪽 셀에서 다시 참조될 예정
print(embedding_layer.weight)


`Embedding Layer`는 본질적으로 **룩업 테이블(look-up table)**과 같은 역할을 합니다.  
위에서 생성한 임베딩 레이어의 생성자에서 `"num_embeddings"`는 vocabulary의 크기를 의미합니다.  
우리 예시에서는 `Vocabulary`의 크기가 20이므로,  
이 레이어는 총 20개의 임베딩 벡터를 저장하고 있습니다:


In [ ]:
print(f'There are {len(vocabulary)} unique tokens in the vocabulary.\n')
print(f'The unique tokens are: \n{vocabulary.tokens()}')

`"num_embeddings"`는 몇 개의 벡터를 초기화할지를 의미합니다.  
우리는 총 20개의 고유한 토큰을 가지고 있으므로, 각 토큰마다 하나씩 총 20개의 벡터가 필요합니다.  
그래서 이 예제에서는 `"num_embeddings"` 값이 20으로 설정되어 있습니다.

한편, `"embedding_dim"`은 각 임베딩 벡터의 차원 수를 의미합니다.  
여기서는 시각적으로 보기 쉽게 하기 위해 임의로 5차원으로 설정한 것입니다.


In [ ]:
# 브로모에테인의 토큰 리스트를 정수 인덱스 리스트로 변환하고 텐서로 감쌈
# (shape을 맞추기 위해 2D 텐서로 만듦: [1, 시퀀스 길이])
numerical_indices_bromoethane = torch.LongTensor([vocabulary.encode(tokenized_bromoethane)])

# 정수 인덱스 확인
print(f"Numerical indices of bromoethane:\n {numerical_indices_bromoethane}\n")

# 임베딩 레이어에 정수 인덱스를 입력하여 임베딩 벡터 출력
embedding = embedding_layer(numerical_indices_bromoethane)

# 각 토큰에 대응하는 임베딩 벡터 확인
print(f"Embedding:\n {embedding}")

`Embedding Layer`는 `"num_embeddings"` 값에 따라 20개의 임베딩 벡터를 가지고 있도록 정의되어 있습니다.  
브로모에테인의 정수 인덱스 시퀀스를 보면, "C"는 인덱스 18번, "Br"은 인덱스 17번에 해당합니다.  
`Embedding Layer`는 이 인덱스들을 기반으로 해당 위치의 임베딩 벡터를 반환합니다.

출력된 임베딩 텐서를 보면, **앞의 두 행은 서로 동일하며**, 이는 "C"가 두 번 반복되었기 때문입니다.  
마지막 토큰인 "Br"은 인덱스 17번이므로 다른 벡터를 가지며,  
위에서 확인했던 17번과 18번 인덱스의 임베딩 벡터와 정확히 일치함을 확인할 수 있습니다.

또한, 각각의 임베딩 벡터의 차원 수는 `"embedding_dim"` 값으로 지정한 대로 **5차원**입니다.

---

**Note:**  
`Embedding Layer`는 무작위로 초기화되므로, 현재 임베딩 벡터의 값 자체는 학습 전까지는 의미가 없습니다.  
이 벡터들은 RNN 훈련 과정 중에 함께 업데이트되며 점점 더 의미 있는 표현을 학습하게 됩니다.

---

**추가로:**  
예제 SMILES 데이터셋에 포함된 "버키볼(bucky ball)"을 기억해봅시다.  
이전에 언급했듯이, **"희귀한(rare)" 토큰들이 Vocabulary 크기를 불필요하게 키울 수 있으며**,  
이는 모델 학습에 부정적인 영향을 줄 수 있습니다.

이처럼 Vocabulary 크기가 커지면 `Embedding Layer`의 `"num_embeddings"` 값도 커지게 됩니다.  
그 결과, RNN이 학습 시 **업데이트해야 할 파라미터 수도 증가**하고,  
계산량 역시 더 많아집니다.  

또한 파라미터 수가 많아질수록 GPU 메모리 사용량도 증가하게 되며,  
메모리가 부족할 경우에는 학습 배치(batch) 크기를 줄여야 하고,  
이는 곧 모델의 학습 시간이 늘어나는 결과를 초래할 수 있습니다.


### 순환 계층 (Recurrent Layers)

`Embedding Layer`는 RNN의 입력이 되는 벡터를 생성합니다.  
이 벡터는 `Recurrent Layer`로 전달되며,  
해당 계층에서는 `Formal Problem Set-up` 섹션에서 설명한 것처럼 **순환 연산(recurrent computation)**이 수행됩니다.

아래 코드는 `Long Short-Term Memory (LSTM)` 셀을 이용하여 `Recurrent Layer`를 초기화하는 예시입니다.  
LSTM은 `자연어 처리(Natural Language Processing, NLP)` 분야에서 매우 뛰어난 성능을 보이며 널리 사용되어 왔습니다.  
LSTM의 자세한 구조와 연산 원리는 다음 원문을 참고하세요:  
https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.676.4320&rep=rep1&type=pdf


In [ ]:
# 여기서는 설명을 위한 목적이므로 LSTM 레이어를 단 1층만 초기화함
# input_size = 5 → 앞서 정의한 Embedding layer의 "embedding_dim"이 5였기 때문
# hidden_size = 5 → 설명과 시각화를 쉽게 하기 위해 임의로 설정한 값
recurrent_layer = nn.LSTM(input_size=5,
                          hidden_size=5,
                          num_layers=1,
                          dropout=0,
                          batch_first=True)  # 입력 텐서의 첫 번째 차원이 배치(batch)임을 명시

# 앞서 만든 임베딩된 SMILES 시퀀스의 shape을 다시 확인 (recurrent layer의 입력으로 사용됨)
# 텐서 shape: (batch size) x (sequence length) x (embedding dimension)
# 여기서는 하나의 SMILES만 사용하므로 배치 크기는 1
embedding.shape


In [ ]:
# 임베딩 벡터를 순환 계층(LSTM)에 통과시킴
# 출력은 두 가지:
# - rnn_output: 각 time-step에서의 출력(hidden state)들의 시퀀스
# - hidden_state: 마지막 time-step의 hidden state (예: 다음 계층으로 전달될 수 있음)
# - cell_state: LSTM 내부의 상태값 (long-term memory 역할)

rnn_output, (hidden_state, cell_state) = recurrent_layer(embedding)


RNN의 출력값은 이후 모델의 다음 단계로 전달되는 벡터입니다.  
hidden state와 cell state는 각각 이전 시점의 정보를 기억하는 역할을 하며,  
즉 지금까지 관찰된 토큰 시퀀스를 추적하는 데 사용됩니다.  
이 둘의 구체적인 역할은 LSTM 셀의 내부 구조와 관련되어 있지만, 여기서는 생략합니다.

이제 출력값의 차원이 올바른지 확인해보겠습니다.


In [ ]:
# rnn_output의 shape은 embedding과 동일하게 나와야 함:
# (배치 크기) x (시퀀스 길이) x (임베딩 차원 또는 hidden size)
print(rnn_output.shape)

# 실제로 임베딩과 동일한 shape인지 확인 (True이면 동일함)
rnn_output.shape == embedding.shape


입력이 `Recurrent Layer`를 통과한 후 (실제 모델에서는 총 3개의 순환 계층이 사용됨),  
그 출력은 마지막 `Linear Layer`로 전달됩니다.  

`Linear Layer`의 목적은 `Recurrent Layer`의 출력을 다음과 같은 차원으로 변환하는 것입니다:

- 입력: (배치 크기) x (시퀀스 길이) x **(hidden size)**  
- 출력: (배치 크기) x (시퀀스 길이) x **(vocabulary 크기)**

출력의 마지막 차원이 vocabulary 크기라는 것은,  
**vocabulary에 포함된 각 토큰마다 하나의 숫자 값**이 있다는 뜻입니다.  
이 숫자들은 모델이 다음 time step에서 **각 토큰을 생성할 "가능성"**을 나타내는 데 사용됩니다.

즉, vocabulary의 각 토큰에 대해  
**"다음에 생성될 토큰으로 선택될 확률"**을 하나씩 부여하게 되는 것이며,  
이 확률 분포는 softmax를 통해 계산됩니다.

이제 multinomial 확률 분포(multinomial distribution)에 대해 논의하기에 앞서,  
`Linear Layer`의 출력을 먼저 확인해보겠습니다.


In [ ]:
# Linear layer를 초기화
# in_features = 5 → 위에서 정의한 recurrent layer의 hidden_size
# out_features = 20 → vocabulary의 크기 (각 time step마다 20개의 토큰 중 하나를 예측해야 함)
linear_layer = nn.Linear(in_features=5,
                         out_features=20)

# RNN의 출력(rnn_output)을 Linear layer에 통과시켜 최종 출력 생성
linear_output = linear_layer(rnn_output)

# 최종 출력의 shape 확인:
# (배치 크기) x (시퀀스 길이) x (vocabulary 크기)
linear_output.shape


### 소프트맥스(Softmax): 다항 조건부 확률 분포(multinomial conditional probability distribution) 얻기

이제 우리는 텐서의 형태가 **(배치 크기) x (시퀀스 길이) x (vocabulary 크기)**인 출력을 가지고 있습니다.  
마지막 단계는 여기에 `Softmax 함수`를 적용하여 **다항 확률 분포(multinomial distribution)**를 얻는 것입니다.  
이 확률 분포는 **다음 time step에서 각 토큰이 생성될 확률**로 해석됩니다.

먼저 `Softmax 함수`의 정의를 소개합니다:

$$
Softmax(token_i) = \frac{e^{token_i}}{\sum\limits_{j=1}^N e^{token_j}}
$$

여기서 $i$, $j = 1, 2, 3, ..., N$은 토큰 인덱스를 의미하며,  
$N$은 vocabulary에 포함된 마지막 토큰 인덱스를 의미합니다.  

`Softmax 함수`는 각 토큰에 대해 **0과 1 사이의 확률 값**을 할당하며,  
전체 토큰 확률의 **합은 1**이 되도록 보장합니다.  
즉, 어떤 토큰이든 **반드시 하나가 선택**되어야 하므로, 전체 확률의 합은 1입니다.

---

**문제점:**  
`Softmax 함수`는 수치적으로 불안정(numerically unstable)할 수 있습니다.  
특히, 지수 함수($e^x$)가 사용되기 때문에  
- 어떤 토큰은 **너무 작은 확률**이 되어 0에 가까워지거나  
- 어떤 토큰은 **너무 큰 확률**이 되어 **오버플로우(overflow)**가 발생할 수 있습니다.

> 💡 **오버플로우란?**  
> 너무 큰 숫자를 계산하려 할 때, 컴퓨터의 수치 표현 한계를 초과하여  
> 무한대(`inf`) 또는 에러가 발생하는 현상입니다.  
> 예를 들어 $e^{1000}$ 같은 값은 컴퓨터가 표현할 수 있는 범위를 넘어설 수 있습니다.


---

이러한 문제를 피하기 위해, 실제 모델에서는 `Softmax` 대신 `Log-Softmax 함수`를 사용합니다.  
이 함수는 확률을 **로그 확률**로 변환하여 출력값의 범위를 $[-\infty, 0)$로 제한합니다.  
이렇게 하면 수치적으로 더 안정적일 뿐만 아니라,  
***확률의 순서(order of magnitudes)***도 그대로 유지됩니다.  
즉, Softmax에서 더 낮은 확률을 갖는 토큰은 Log-Softmax에서도 더 낮은 값을 가지게 됩니다.

`Log-Softmax` 함수는 다음과 같이 정의됩니다:

$$
Log\text{-}Softmax(token_i) = \log\left(\frac{e^{token_i}}{\sum\limits_{j=1}^N e^{token_j}}\right)
$$

---

이제 위에서 얻은 `Linear Layer`의 출력에 `Log-Softmax 함수`를 적용해 보겠습니다.


In [ ]:
# 먼저 일반 softmax 출력을 확인
# linear layer의 출력은 (배치 크기) x (시퀀스 길이) x (vocabulary 크기)의 형태를 가지고 있음
# 따라서 dim=2로 설정하면, 각 시퀀스 위치마다 vocabulary 차원에 대해 softmax를 적용함
# 즉, 각 토큰에 대한 확률 값을 얻을 수 있음
softmax = linear_output.softmax(dim=2)
print(softmax)

# softmax의 장점: 각 시퀀스 위치에서 모든 토큰 확률의 합이 항상 1이 됨
# 아래는 그 합을 확인하는 코드
print(softmax.sum(dim=2))


In [ ]:
# 이제 log-softmax 출력을 확인해보자
# softmax와 동일하게 vocabulary 차원(dim=2)에 대해 log-softmax를 적용
log_softmax = linear_output.log_softmax(dim=2)
print(log_softmax)

# log-softmax는 확률이 아니라 로그 확률이므로
# 전체 값을 더해도 합이 1이 되지 않음 (오히려 음수의 합이 됨)
# log-softmax는 값의 범위가 [-inf, 0) 이기 때문
print(log_softmax.sum(dim=2))


**마지막으로, `Log-Softmax 함수`의 출력은 어떻게 토큰 확률로 해석될까요?**

각 텐서는 다음 time step에서 **vocabulary에 포함된 각 토큰이 생성될 확률**을 나타냅니다.  
보다 구체적으로 설명하면:


In [ ]:
# 원래 입력으로 사용한 브로모에테인의 SMILES 문자열을 다시 출력
print(f"Original SMILES string: {bromoethane_smiles}\n")

# vocabulary에 포함된 고유한 토큰 목록을 출력
print(f"The unique tokens are: \n{vocabulary.tokens()}\n")

# 위에서 얻은 log-softmax 출력에서 각 time step마다 가장 확률이 높은 토큰의 인덱스를 추출
most_probable_tokens = log_softmax.argmax(dim=2).flatten().tolist()

# 각 시점(time step)마다 예측된 토큰과 정답 토큰을 비교하여 출력
for idx, (correct_token, most_probable_token) in enumerate(zip(tokenized_bromoethane, most_probable_tokens)):
    print(f"At time step {idx+1}, the generative model proposes {vocabulary.tokens()[most_probable_token]} as the most probable token and the correct token is {correct_token}")


**두 셀 위에서 출력된 log-softmax 결과를 보면, 각 토큰에 대한 확률 값이 매우 유사한 것을 확인할 수 있습니다.  
이는 모델이 아직 학습되지 않았기 때문에, 각 토큰을 생성할 확률이 거의 동일하다고 생각하고 있다는 뜻입니다.  
그 결과, 위에서 확인한 것처럼 모델은 정답 토큰을 잘 맞추지 못하고 있습니다.**

---

### 요약: 생성 모델 샘플링 절차는 다음과 같습니다

1. **SMILES 문자열을 토큰화**한다

2. **토큰을 인코딩**하여 vocabulary 내 정수 인덱스 시퀀스로 변환한다

3. **벡터 정렬(collation)**을 통해 다양한 길이의 SMILES라도  
   모델 입력 차원이 동일하도록 padding 등으로 벡터를 정규화한다

4. **임베딩 벡터**를 생성하여 RNN에 입력할 수 있는 벡터 형태로 만든다

5. **임베딩 벡터를 RNN에 전달**하여 순차적으로 정보를 처리한다

6. **RNN의 출력에 선형 변환(linear transformation)**을 적용하여  
   출력의 마지막 차원을 vocabulary 크기와 맞춘다

7. **Log-softmax**를 적용하여 각 토큰이 다음 time step에서 생성될 확률을 나타내는  
   **다항 확률 분포(multinomial token probability distribution)**를 얻는다


## 모델은 어떻게 학습하는가? (How Does The Model Learn?)

앞선 섹션에서는 **원시 SMILES 문자열이 모델을 통해 어떻게 처리되는지**를 단계별로 살펴보았습니다.  
그 결과로 얻은 **다항 확률 분포(multinomial probability distribution)**는  
모든 토큰이 거의 동일한 확률로 예측되는 등, 학습되지 않은 상태에서는 유용하지 않다는 점을 확인했습니다.

이번 섹션에서는 모델이 학습을 통해 어떻게 개선되는지를 설명하며,  
이를 위해 사용하는 손실 함수인 **Negative Log-Likelihood (NLL)**을 소개합니다.

손실 함수는 다음과 같이 정의됩니다:

$$
NLL = -\log(probability_{token})
$$

`Log-Softmax 함수`는 이미 로그(log)를 취한 값을 반환하므로,  
`Negative Log-Likelihood`는 단순히 `Log-Softmax` 출력의 부호를 반전시키는 것과 같습니다.

이제 이 손실 함수가 실제 텐서 상에서 어떻게 적용되는지를 살펴보겠습니다:


In [ ]:
# 손실 함수(loss function)로 Negative Log-Likelihood (NLLLoss)를 정의
# reduction='none'으로 설정하면 각 시점(time step)마다 개별 손실값을 반환함
loss = torch.nn.NLLLoss(reduction='none')

# "CCBr" SMILES에 대해 모델이 예측해야 하는 정답 토큰 인덱스를 다시 출력
print(f"These are the token indices we would want our model to predict:\n{numerical_indices_bromoethane}\n")

# log-softmax 출력도 다시 확인
print(f"Recall the log-softmax output:\n{log_softmax}\n")

# NLLLoss는 입력 텐서의 shape이 (배치 크기) x (클래스 수) x (시퀀스 길이) 형태여야 하므로
# 두 번째와 세 번째 차원을 바꾸기 위해 transpose 수행
print(f"We will transpose the log-softmax tensor to have shape (batch size) x (vocabulary) x (sequence length):\n{log_softmax.transpose(1,2)}\n")

# 최종적으로 NLLLoss에 log-softmax 결과와 정답 인덱스를 넣어 손실값 출력
# 출력 텐서는 각 시점마다의 개별 손실값을 포함함
print(f"The output tensor from negative log-likelihood is:\n{loss(log_softmax.transpose(1, 2), numerical_indices_bromoethane)}\n")


우리가 time step 1에서 예측하고자 했던 정답 토큰 인덱스는 **18**이었습니다.  
log-softmax 텐서를 transpose한 결과의 첫 번째 column에서 18번째 인덱스(뒤에서 두 번째 row)를 확인해보면,  
`Negative Log-Likelihood`는 해당 값의 **부호를 단순히 반전시킨 것**임을 알 수 있습니다.  

마찬가지로, 마지막 time step에서는 예측해야 할 정답 토큰 인덱스가 **17**이었습니다.  
세 번째 column에서 17번째 인덱스(뒤에서 세 번째 row)를 확인해보면,  
역시 해당 값의 음수가 `Negative Log-Likelihood`로 사용된 것을 확인할 수 있습니다.

즉, `Negative Log-Likelihood`는 모델이 정답 토큰에 부여한 **log 확률(log-softmax 값)**에 대해  
그 값을 **음수로 바꿔서 손실(loss)**로 계산합니다.  
이 손실 값이 클수록 모델은 정답 토큰에 낮은 확률을 부여했다는 의미이며,  
손실 값이 작을수록 올바른 토큰에 높은 확률을 준 것이므로 성능이 좋은 상태입니다.

다시 말해, 우리는 모델이 정답 토큰에 가능한 한 **높은 확률**을 할당하기를 원하므로,  
모델 학습의 목표는 이 `Negative Log-Likelihood`를 **최소화(minimize)** 하는 것입니다.

---

마지막으로, 역전파(backpropagation) 과정에서는  
`Negative Log-Likelihood` 텐서의 값을 **모두 합산하여 전체 손실**을 계산합니다.  
이렇게 얻은 총 손실 값이 현재 SMILES 시퀀스에 대해 모델이 얼마나 잘못 예측했는지를 나타냅니다.


In [ ]:
loss(log_softmax.transpose(1, 2), numerical_indices_bromoethane).sum(dim=1)

## 학습된 모델은 어떤 모습일까? (What Does a Trained Model Look Like?)

이전 섹션에서는 손실 함수가 어떻게 계산되는지를 살펴보았고,  
학습되지 않은 모델이 모든 토큰에 대해 거의 동일한 확률을 부여한다는 것도 확인했습니다.  
이번 섹션에서는 **학습이 완료된 모델이 어떻게 동작하는지를 수치적으로** 보겠습니다.

우리가 궁극적으로 달성하고자 하는 목표는  
**SMILES 데이터셋의 내재된 확률 분포(underlying probability distribution)**를 모델이 학습하는 것입니다.

이를 위해, `ZINC` 데이터셋으로부터 SMILES 데이터를 다운로드하고, 분자 생성 모델을 학습시킵니다.  
사용된 스크립트는 다음과 같습니다:

- `01_download_data.py`  
- `02_train_model_locally.py`  

이번 예제에서는 총 50,000개의 SMILES 문자열을 사용하여  
50 에폭(epoch) 동안 모델을 학습하였습니다.

> ⚠️ **주의:** 모델을 학습할 때는 반드시 validation 세트에서의 손실(loss)을 모니터링해야 합니다.  
> validation loss가 증가하기 시작하면 이는 학습 데이터에 과적합(overfitting)되고 있다는 신호일 수 있습니다.

이번 섹션의 목적은 학습된 모델이 ***어떻게*** 동작하는지를 보여주는 것이므로,  
학습 횟수는 임의로 50 에폭으로 설정하였습니다.

학습이 완료된 모델은 저장소에 포함되어 있으며, 다음 파일로 제공됩니다:  
`model.49.pth`


In [ ]:
# 훈련된 모델 로드
from smiles_lstm.model.smiles_lstm import SmilesLSTM
from utils import load_from_file

trained_model = load_from_file("pretrained.rnn.pth")

# Vocabulary 확인
print(f"There are {len(trained_model.vocabulary)} unique tokens:\n{trained_model.vocabulary.tokens()}")

Vocabulary가 훨씬 커졌습니다!  
이는 상당수의 토큰이 **입체화학(Stereochemistry)** 정보를 담고 있기 때문이며,  
이는 원본 `ZINC` 학습 데이터에 **입체이성질체(Stereoisomers)**가 포함되어 있기 때문입니다.

실제로는 이러한 입체 이성질체 정보가 모델의 복잡도를 크게 증가시킬 수 있으므로,  
모델을 학습하기 전에 입체이성질체를 **제거(preprocessing)**하는 것이 유리할 수 있습니다.  
하지만 이 예제에서는 모델이 어떻게 작동하는지를 보여주는 것이 목적이므로,  
데이터셋 필터링은 생략하였습니다.

---

이제 앞서 제시했던 벤젠(benzene) 생성 예제를 다시 사용해보겠습니다.  
SMILES 시퀀스:

**c1ccccc1**

이 중 **"c1ccccc"**까지 입력되었을 때,  
학습된 모델은 다음 토큰으로 **"1"**을 높은 확률로 예측하여  
벤젠 고리를 완성해야 합니다.

이제 실제로 이것이 어떻게 나타나는지 확인해보겠습니다:


In [ ]:
# collate_fn 메서드를 사용하기 위해 Dataset 모듈에서 불러옴
from smiles_lstm.model.smiles_dataset import Dataset

# 벤젠 분자 SMILES 시퀀스 (고리 닫히기 전까지)
benzene = 'c1ccccc'

# 1. SMILES 문자열을 토큰화
benzene_tokenized = tokenize(benzene)

# 2. 토큰을 정수 인덱스로 인코딩 (trained model의 vocabulary 사용)
benzene_encoded = trained_model.vocabulary.encode(benzene_tokenized)

# 3. 인코딩된 토큰 리스트를 PyTorch 텐서로 변환 (collate에 입력하기 위함)
benzene_encoded = torch.tensor([benzene_encoded])

# 4. collate_fn을 사용하여 배치 형태로 정렬 (길이 맞춤 및 padding 처리 포함)
benzene_collated = Dataset.collate_fn(benzene_encoded)


In [ ]:
# collated된 입력 텐서를 학습된 모델에 전달
# [:, :-1]는 마지막 토큰을 제외한 입력 시퀀스를 사용 (다음 토큰 예측을 위한 입력)
logits, _ = trained_model.network(benzene_collated[:, :-1])

# 모델 출력에 log-softmax를 적용하여 각 위치에서의 토큰 확률 분포(log 확률)를 얻음
log_probs = logits.log_softmax(dim=2)

# 벤젠 시퀀스의 마지막 토큰(c1ccccc) 이후, 다음 토큰을 예측하는 확률 분포를 추출
# [0][7]은 첫 번째 시퀀스의 8번째 위치(time step 8)에 대한 로그 확률 벡터를 의미함
log_probs[0][7]


위 텐서의 의미는 다음과 같습니다:  
**지금까지 관찰된 시퀀스 c1ccccc에 기반하여, 다음에 생성될 토큰들의 확률 분포**

이전의 학습되지 않은 모델과는 확연한 차이가 있습니다.  
이제는 각 토큰에 대해 **서로 다른 확률 값**이 할당되어 있으며,  
이는 모델이 시퀀스를 기반으로 학습되었음을 보여줍니다.

이제 우리는 이 분포에서 **가장 높은 확률을 가진 토큰**,  
즉 **가장 생성될 가능성이 높은 토큰**을 찾아보겠습니다:


In [ ]:
# log 확률 벡터
log_prob_vector = log_probs[0][7]

# log 확률 기준 상위 두 개 인덱스 추출
topk_indices = torch.topk(log_prob_vector, k=3).indices.tolist()

for rank, idx in enumerate(topk_indices, start=1):
    token = trained_model.vocabulary.tokens()[idx]
    log_prob = log_prob_vector[idx].item()
    prob = torch.exp(log_prob_vector[idx]).item()
    print(f"[Top {rank}] Token: {token}")
    print(f"         Log-Prob: {log_prob:.3f}")
    print(f"         Prob:     {prob:.3f}\n")


가장 높은 확률을 가진 토큰은 **"1"**이며, 이는 정답 토큰입니다!  
이 토큰을 시퀀스에 추가하면 벤젠(Benzene)이 완성됩니다:

**c1ccccc1**

즉, 모델이 실제로 무언가를 학습했다는 것을 확인할 수 있습니다! 🎉

---

### 학습 데이터의 영향 (Training Data Implication)

이 섹션을 마무리하며 중요한 점 하나를 강조하고자 합니다.  
**RNN은 학습 데이터의 확률 분포를 그대로 학습**하므로,  
학습에 사용한 데이터의 ***분자 특성***이  
모델이 생성할 ***분자의 특성***에 큰 영향을 줍니다.

좀 더 구체적으로 말하자면,  
RNN은 일반적으로 ChEMBL이나 ZINC와 같은 데이터베이스로 학습됩니다.  
이들 데이터베이스에는 **"약물 유사성(drug-like)"을 가진 분자**들이 수록되어 있습니다.  
따라서 학습된 RNN 역시 대체로 약물 유사한 분자들을 생성하게 됩니다.

여기서 "약물 유사성(drug-likeness)"은 보통 **Lipinski's Rule of Five**를 만족하는지로 판단합니다.  
이 규칙은 분자의 용해도(solubility), 분자량, 수소 결합 수용체/공여체 수 등  
일정한 특성 범위를 기준으로 설정되어 있습니다.

> 💡 단, **Lipinski 규칙은 절대적인 기준이 아닌 가이드라인**이라는 점에 주의하세요.  
> 보다 자세한 내용은 원 논문을 참고하세요:  
> https://www.sciencedirect.com/science/article/abs/pii/S0169409X96004231


## 학습된 모델로부터 SMILES 샘플링하기 (Trained Model: Sampling SMILES)

이 마지막 섹션에서는 학습된 모델로부터 **SMILES를 샘플링(sampling)**하는 과정을 소개하고,  
`Likelihood`에 대한 실용적인 해석도 함께 설명합니다.

아래 코드는 `smiles_lstm.py` 파일에서 발췌된 것입니다:


In [ ]:
# 배치 크기를 3으로 고정하여 3개의 SMILES를 샘플링하는 함수
def sample(batch_size=3):
    # 시작 토큰 초기화 → vocabulary에서 "^" 토큰에 해당하는 인덱스를 각 시퀀스의 시작으로 설정
    start_token = torch.zeros(batch_size,
                              dtype=torch.long,
                              device='cpu')
    start_token[:] = trained_model.vocabulary["^"]

    # 현재 입력 벡터를 시작 토큰으로 설정
    input_vector = start_token

    # 샘플링 시퀀스를 저장할 리스트 초기화 (첫 토큰은 "^"으로 구성된 [batch_size, 1] 형태의 텐서)
    sequences = [
        trained_model.vocabulary["^"] * torch.ones([batch_size, 1],
                                                   dtype=torch.long,
                                                   device='cpu')
    ]

    # 주의: 루프 안에서 첫 번째 토큰은 추가되지 않기 때문에
    # 시퀀스를 시작 토큰으로 미리 초기화해놓음

    # RNN의 초기 hidden state (None으로 설정하면 내부적으로 자동 초기화됨)
    hidden_state = None

    # 각 시퀀스에 대한 누적 음의 로그 가능도(Negative Log-Likelihood)를 저장할 텐서
    nlls = torch.zeros(batch_size)

    # 최대 255개의 토큰을 샘플링 (start 토큰 포함하여 총 길이 256까지)
    for _ in range(256 - 1):
        # 현재 토큰을 모델에 입력 (unsqueeze로 shape을 [batch, 1]로 맞춤)
        logits, hidden_state = trained_model.network(input_vector.unsqueeze(1),
                                                     hidden_state)

        # 출력 shape을 [batch_size, vocab_size]로 맞추기 위해 squeeze
        logits = logits.squeeze(1)

        # softmax를 사용하여 확률 분포 계산
        probabilities = logits.softmax(dim=1)

        # log-softmax도 함께 계산하여 NLL에 사용
        log_probs = logits.log_softmax(dim=1)

        # 확률 분포로부터 다음 토큰을 다항 샘플링 (sampling 기반 생성)
        input_vector = torch.multinomial(probabilities, 1).view(-1)

        # 시퀀스에 샘플링된 토큰 추가
        sequences.append(input_vector.view(-1, 1))

        # 현재 time-step에서의 NLL 값을 누적
        nlls += loss(log_probs, input_vector)

        # 모든 시퀀스가 종료 토큰("$")을 생성했다면 조기 종료
        if input_vector.sum() == 0:  # "$" 토큰의 인덱스가 0이라고 가정
            break

    # 생성된 모든 토큰들을 [batch_size, seq_length] 형태로 결합
    sequences = torch.cat(sequences, 1)

    # 최종적으로 샘플링된 시퀀스와 각 시퀀스의 누적 NLL 값을 반환
    return sequences.data, nlls


이제 몇 가지 핵심 단계들을 하나씩 살펴보겠습니다:


In [ ]:
# 배치 크기(여기서는 3)에 해당하는 길이의 텐서를 생성하고, 모든 값은 0으로 초기화
# 이 텐서의 각 원소는 하나의 SMILES 시퀀스의 시작을 나타냄

start_token = torch.zeros(3,
                          dtype=torch.long,
                          device='cpu')

# 각 시퀀스의 시작 위치에 "시작 토큰"("^")의 vocabulary 인덱스를 할당
# 즉, 모델이 시퀀스를 생성할 때 시작 지점임을 명시
start_token[:] = trained_model.vocabulary["^"]

# 시작 토큰 텐서 출력
start_token # torch.Size([3])


In [ ]:
# 시작 토큰을 포함하는 입력 벡터 초기화
# 이 벡터는 첫 타임스텝에서 RNN에 입력될 값이며, 모든 시퀀스는 "^"로 시작함
input_vector = start_token

# 입력 벡터를 모델에 전달
# 내부적으로는: 임베딩 → RNN → linear layer 순으로 처리됨
# unsqueeze(1): shape이 [3]인 1차원 텐서를 [3, 1]로 변환
# 이는 RNN이 기대하는 입력 형식인 [batch_size, sequence_length]에 맞추기 위한 조치임
# 즉, 각 시퀀스를 길이 1짜리 시퀀스로 인식시켜 3개의 시퀀스 배치로 처리하게 함
logits, hidden_state = trained_model.network(input_vector.unsqueeze(1))

# 출력 텐서의 shape을 [batch_size, vocab_size]로 맞추기 위해 sequence 차원을 제거
logits = logits.squeeze(1)


지금까지 수행한 연산들은 앞선 섹션에서 소개한 내용과 동일합니다.  
하지만 다음 단계는 **SMILES를 샘플링할 때의 핵심적인 차이점**입니다.  
이제 우리는 **토큰 확률을 다항 분포(multinomial distribution)**로 **명시적으로** 다루게 됩니다.


In [ ]:
# 여기서는 일반적인 softmax 함수를 적용하여
# 각 시퀀스에서 다음 토큰으로 선택될 확률 분포를 계산함
# dim=1: vocabulary 차원에 대해 softmax를 적용하여
# 모든 토큰의 확률 합이 1이 되도록 함
probabilities = logits.softmax(dim=1)


앞에서 `Log-Softmax 함수`가 수치적으로 더 안정적인 계산 방법이라고 설명했는데,  
그렇다면 왜 여기서는 다시 일반적인 `Softmax 함수`를 사용하는 걸까요?

이는 용도가 다르기 때문입니다.

`Log-Softmax 함수`는 학습 과정에서 `Negative Log-Likelihood` 손실을 최소화할 때  
**수치적 안정성(numerical stability)**을 확보하기 위해 사용됩니다.

반면, `Softmax 함수`는 **모든 토큰의 확률 합이 1이 되는** 성질이 있기 때문에  
출력 텐서를 **다항 확률 분포(multinomial probability distribution)**로 해석할 수 있습니다.

이렇게 얻은 확률 분포를 기반으로 우리는 실제로 **다음 토큰을 샘플링**할 수 있습니다.  
즉, 이 시점에서는 예측 성능을 계산하려는 게 아니라  
***생성(generation)***을 위한 확률 분포로 활용하는 것이므로 `Softmax`가 사용됩니다.


In [ ]:
# torch.multinomial 함수는 softmax로부터 얻은 확률 분포를
# 다항 분포(multinomial distribution)로 간주하고,
# 각 시퀀스(batch)마다 확률에 따라 토큰을 1개 샘플링함
input_vector  = torch.multinomial(probabilities, 1).view(-1)


이제 하나의 시퀀스에 대해,  
각 토큰의 확률 분포와 샘플링된 토큰을 확인해보겠습니다:


In [ ]:
# 첫 번째 시퀀스(batch 0)에 대해, 모든 토큰에 대한 softmax 확률 출력
print(f"All token probabilities:\n{probabilities[0]}\n")

# 확률이 가장 높은 토큰의 인덱스를 출력 (가장 가능성이 높은 토큰)
print(f"The token with the highest probability:\n{torch.argmax(probabilities[0])}\n")

# 다항 분포에서 실제로 샘플링된 토큰의 인덱스를 출력
print(f"The sampled token:\n{input_vector[0]}")


가장 높은 확률을 가진 토큰이 항상 샘플링되는 것은 아님을 확인할 수 있습니다.  
이처럼 샘플링에는 **확률적 요소**가 있기 때문에,  
softmax 분포에서 높은 확률을 가진 토큰이 선택되지 않을 수도 있습니다.

마지막 단계에서는 지금까지 생성된 시퀀스를 저장하고,  
`Negative Log-Likelihood (NLL)`를 계산합니다.  
이 과정은 앞서 설명한 `모델은 어떻게 학습하는가? (How Does The Model Learn?)` 섹션과 동일합니다.

이제 `sample` 메서드를 실행하여 실제로 생성된 SMILES를 확인해보겠습니다:


In [ ]:
# sample() 메서드를 실행하여 SMILES 시퀀스를 생성하고, 각 시퀀스의 NLL도 함께 반환받음
sequences, nlls = sample()

# 생성된 시퀀스를 SMILES 문자열로 변환
# 1. 정수 인덱스 → 토큰 (decode)
# 2. 토큰 리스트 → 문자열 (untokenize)
smiles = [trained_model.tokenizer.untokenize(trained_model.vocabulary.decode(seq)) for seq in sequences.numpy()]

# 최종적으로 생성된 SMILES 문자열들을 출력
smiles


In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw

# SMILES 문자열들을 RDKit의 Mol 객체로 변환
# 유효하지 않은 SMILES는 None이 될 수 있음
molecules = [Chem.MolFromSmiles(s) for s in smiles]

# 변환된 Mol 객체들을 격자(grid) 형태로 시각화
# 참고: 유효하지 않은 SMILES가 포함된 경우, 3개 미만의 분자가 출력될 수 있음
# 그럴 경우 위의 샘플링 셀을 다시 실행하여 새로운 SMILES를 생성해볼 수 있음
Draw.MolsToGridImage(molecules)


학습된 생성 모델이 실제로 **유효한(valid) SMILES 문자열**을 생성할 수 있다는 것을 확인했습니다!  
물론 생성된 SMILES가 실제로 바람직한 분자인지는 오늘 강의의 범위를 벗어나지만,  
**SMILES 데이터셋의 확률 분포를 학습한 모델이 SMILES 문법을 익히고  
유효한 분자 구조를 생성할 수 있다**는 점을 잘 보여줍니다.

---

이제 마지막으로 살펴볼 주제는 `Negative Log-Likelihood`입니다.  
이 용어는 간단히 `Likelihood`라고 불리기도 합니다.


In [ ]:
# 위에서 생성된 3개의 SMILES 분자 각각에 대한
# 누적 Negative Log-Likelihood(NLL) 값을 출력
# 이 값은 모델이 해당 시퀀스를 생성하는 데 얼마나 "자신 있었는지"를 나타냄
nlls


### `Negative Log-Likelihood`의 의미는 무엇일까?

다음 세 가지를 기억해 봅시다:

1. `Negative Log-Likelihood`는 전체 시퀀스에서 각 토큰이 생성될 확률을 모두 합산한 값입니다.

2. 이 계산은 `Log-Softmax 함수`의 출력을 기반으로 수행됩니다.

3. `Log-Softmax 함수`의 출력 범위는 $[-\infty, 0)$입니다.

이 말은 곧, 특정 토큰이 생성될 **로그 확률이 클수록(0에 가까울수록)**  
그 토큰이 생성될 가능성이 높다는 것을 의미합니다.  
**따라서 `Negative Log-Likelihood` 값이 작을수록, 해당 시퀀스가 생성될 확률이 크다는 뜻입니다.**

예를 들어 위에서 생성된 예시에서,  
모델은 1번보다 2번과 3번 분자를 생성할 가능성이 더 높다고 말할 수 있습니다.

> 💡 `Negative Log-Likelihood`는 종종  
> **"해당 SMILES 시퀀스를 생성할 가능도(Likelihood)"**라고 표현되기도 합니다.


## 실제 응용에서의 Negative Log-Likelihood

분자 생성 모델을 사용할 때, 단순히 유효한 SMILES를 생성하는 것을 넘어서  
생성된 SMILES가 ***바람직한 특성***을 갖기를 원하게 됩니다.  
이러한 목적을 달성하는 대표적인 방법 중 하나는 `REINVENT`에서 사용된  
**강화학습(Reinforcement Learning, RL)** 방식이지만,  
이는 이 노트북의 범위를 벗어나므로 여기서는 다루지 않습니다.

---

또 다른 널리 사용되는 방법은 **파인튜닝(Fine-Tuning)** 또는  
**전이학습(Transfer Learning)**입니다.  

이 방법에서는 이미 학습된 생성 모델을 사용하되,  
**SMILES 문법을 익힌 기존 모델을 작은 SMILES 데이터셋(fine-tuning dataset)에 대해  
추가로 학습시킵니다.**  

이 작은 데이터셋은 일반적으로 이미 **바람직한 특성을 가진 SMILES**들로 구성됩니다.  
예: 높은 생물학적 활성(potency)이 확인된 화합물 등

---

이 경우 모델은 해당 데이터셋에 속한 SMILES 또는  
**유사한 SMILES를 더 자주 생성하도록 학습**됩니다.

이러한 학습이 실제로 잘 작동하는지를 보여주는 지표가 바로  
**`Negative Log-Likelihood`의 감소입니다.**

즉, 파인튜닝 전보다 후에 특정 SMILES에 대해 NLL 값이 작아졌다면,  
모델이 해당 구조를 **더 잘 생성할 수 있게 되었음을 의미**합니다.

---

📌 실제 논문 예시는 다음을 참고하세요:  
**Fig. 1** in [this paper](https://link.springer.com/article/10.1007/s10822-021-00392-8)
